In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import networkit as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import phate

from scipy import sparse

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from IPython.display import display, Math, Latex, Markdown
from tqdm.notebook import tqdm

from External.ICT.calculate_ICT import calculate_ICT, calculate_sub_ICTs
from External.clustering import centers, k_means_pp
from External.generation import create_graph
from External.plotting import plot_points, plot_graph, no_intersections
from External.create_k_nearest import patch_together
from copy import deepcopy

plt.style.use('standard.mplstyle')

In [5]:
import numpy as np
from scipy import ndimage
from skimage.morphology import medial_axis
import matplotlib.pyplot as plt

In [6]:
# Hyperparameters
mode = "K_Nearest"
metric = "euclidean"
dataset = "non_convex"
ICT_algorithm = "cluster_all"


# image loading
n = number_of_nodes = 1000
Random = False
image_name = "image"

In [7]:
def load_image(filename):
    a=plt.imread(filename+'.png')
    rgb_weights = [0.2989, 0.5870, 0.1140]
    
    grayscale_image = np.dot(a[...,:3], rgb_weights)>0
    
    return grayscale_image
    
    
def sample_points_from_image(n,img,Random=True):
    if not Random:
        random.seed(42)
    non_zero = np.where(img!=0)
    # non_zero=np.vstack((non_zero[0],non_zero[1])).T
    
    
    idx = random.sample(range(len(non_zero[0])),n)
    
    x_coord = non_zero[0][idx]
    y_coord = non_zero[1][idx]
    return x_coord,y_coord

In [8]:
# Compute the position array
img = load_image(image_name).astype(int)

In [9]:
image = deepcopy(img)
medial_axis = np.zeros(shape=image.shape)
distance = np.zeros(shape=image.shape)
boundary = np.zeros(shape=image.shape).tolist()

row = image[0]

for i in range(len(image)):
    boundary[i][len(row)-1] = [[i, len(row-1)]]
    boundary[i][0] = [[i, 0]]
    
for j in range(len(row)):
    boundary[0][j] = [[0, j]]
    boundary[len(image)-1][j] = [[len(image), j]]
    

for i, row in enumerate(image):
    for j, pixel in enumerate(row):
        if not pixel:
            boundary[i][j] = [[i, j]]
    

In [ ]:
iteration = 0
while np.sum(image) > 0:
    new_image = np.zeros(shape=image.shape)
    print(iteration)
    for i, row in enumerate(image):
        for j, pixel in enumerate(row):
#             assert image[i,j] == pixel
            if not pixel:
                continue
            
            if (i == 0) or (j == 0) or (i == len(image)-1) or (j == len(row)-1):
                continue

            if (image[i+1,j]) and (image[i-1,j]) and (image[i,j+1]) and (image[i,j-1]):
                new_image[i,j] = 1
                
            else:
                boundary[i][j] = []
                for i_ in [i-1, i+1]:
                    for j_ in [j-1, j+1]:
                        if not image[i_, j_]:
                            for elem in boundary[i_][j_]:
                                boundary[i][j].append(elem)
                
 
                        
            if (image[i+1,j]) + (image[i-1,j]) + (image[i,j+1]) + (image[i,j-1]) <= 2:
                medial_axis[i,j] = 1
                distances = pairwise_distances(boundary[i][j])
                distance[i,j] = np.max(distances)
                    
    image = deepcopy(new_image)
                
    iteration += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [ ]:
plt.imshow(medial_axis.T[::-1], cmap="gray")

In [ ]:
plt.imshow(img.T[::-1], cmap="magma")